# Use Closed-Form Policy to Play FrozenLake8x8-v1

In [1]:
import sys
import logging
import itertools

import numpy as np
np.random.seed(0)
import gym

logging.basicConfig(level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('FrozenLake8x8-v1')
for key in vars(env.spec):
    logging.info('%s: %s', key, vars(env.spec)[key])
for key in vars(env.unwrapped):
    logging.info('%s: %s', key, vars(env.unwrapped)[key])

00:00:00 [INFO] id: FrozenLake8x8-v1
00:00:00 [INFO] entry_point: gym.envs.toy_text:FrozenLakeEnv
00:00:00 [INFO] reward_threshold: 0.85
00:00:00 [INFO] nondeterministic: False
00:00:00 [INFO] max_episode_steps: 200
00:00:00 [INFO] order_enforce: True
00:00:00 [INFO] _kwargs: {'map_name': '8x8'}
00:00:00 [INFO] _env_name: FrozenLake8x8
00:00:00 [INFO] desc: [[b'S' b'F' b'F' b'F' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'F' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'H' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'F' b'F' b'H' b'F' b'F']
 [b'F' b'F' b'F' b'H' b'F' b'F' b'F' b'F']
 [b'F' b'H' b'H' b'F' b'F' b'F' b'H' b'F']
 [b'F' b'H' b'F' b'F' b'H' b'F' b'H' b'F']
 [b'F' b'F' b'F' b'H' b'F' b'F' b'F' b'G']]
00:00:00 [INFO] nrow: 8
00:00:00 [INFO] ncol: 8
00:00:00 [INFO] reward_range: (0, 1)
00:00:00 [INFO] P: {0: {0: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 8, 0.0, False)], 1: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 8, 0.0, 

00:00:00 [INFO] isd: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
00:00:00 [INFO] lastaction: None
00:00:00 [INFO] nS: 64
00:00:00 [INFO] nA: 4
00:00:00 [INFO] action_space: Discrete(4)
00:00:00 [INFO] observation_space: Discrete(64)
00:00:00 [INFO] np_random: RandomState(MT19937)
00:00:00 [INFO] s: 0
00:00:00 [INFO] spec: EnvSpec(FrozenLake8x8-v1)


In [3]:
class ClosedFormAgent:
    def __init__(self, env):
        state_n, action_n = env.observation_space.n, env.action_space.n
        v = np.zeros((env.spec.max_episode_steps+1, state_n))
        q = np.zeros((env.spec.max_episode_steps+1, state_n, action_n))
        pi = np.zeros((env.spec.max_episode_steps+1, state_n))
        for t in range(env.spec.max_episode_steps-1, -1, -1):
            for s in range(state_n):
                for a in range(action_n):
                    for p, next_s, r, d in env.P[s][a]:
                        q[t, s, a] += p * (r + (1. - float(d)) * v[t+1, next_s])
                v[t, s] = q[t, s].max()
                pi[t, s] = q[t, s].argmax()
        self.pi = pi

    def reset(self, mode=None):
        self.t = 0

    def step(self, observation, reward, termination):
        action = self.pi[self.t, observation]
        self.t += 1
        return action

    def close(self):
        pass


agent = ClosedFormAgent(env)

In [4]:
def play_episode(env, agent, seed=None, mode=None, render=False):
    observation, _ = env.reset(seed=seed)
    reward, termination, truncation = 0., False, False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, termination)
        if render:
            env.render()
        if termination or truncation:
            break
        observation, reward, termination, truncation, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.info('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

00:00:01 [INFO] ==== test ====
00:00:01 [INFO] test episode 0: reward = 1.00, steps = 160
00:00:01 [INFO] test episode 1: reward = 1.00, steps = 89
00:00:01 [INFO] test episode 2: reward = 0.00, steps = 195
00:00:01 [INFO] test episode 3: reward = 1.00, steps = 94
00:00:01 [INFO] test episode 4: reward = 1.00, steps = 196
00:00:01 [INFO] test episode 5: reward = 0.00, steps = 200
00:00:01 [INFO] test episode 6: reward = 1.00, steps = 189
00:00:01 [INFO] test episode 7: reward = 1.00, steps = 141
00:00:01 [INFO] test episode 8: reward = 1.00, steps = 103
00:00:01 [INFO] test episode 9: reward = 1.00, steps = 191
00:00:01 [INFO] test episode 10: reward = 1.00, steps = 83
00:00:01 [INFO] test episode 11: reward = 1.00, steps = 120
00:00:01 [INFO] test episode 12: reward = 1.00, steps = 83
00:00:01 [INFO] test episode 13: reward = 1.00, steps = 140
00:00:01 [INFO] test episode 14: reward = 1.00, steps = 70
00:00:01 [INFO] test episode 15: reward = 1.00, steps = 132
00:00:01 [INFO] test epi

In [5]:
env.close()